### 4.1.1 Adding color channels
- There are several ways to encode colors into numbers.
    - **This is something of an understatement**: https://en.wikipedia.org/wiki/Color_model
### 4.1.2 Loading an image file
- Images come in several different file formats, but luckily there are plenty of ways to load images in Python.

In [2]:
import imageio
import torch

In [3]:
img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

(720, 1280, 3)

### 4.1.3 Changing the layout
- We can use the tensor's `permute` method with the old dimensions for each new dimension to get to an appropriate layout.

In [4]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

- Note this operation does not make a copy of the tensor data.
- As a slightly more efficient alternative to using `stack` to build up the tensor, we can preallocate a tensor of appropriate size and fill it with images loaded from a directory:

In [5]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [6]:
import os

In [7]:
data_dir = '../data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir)
            if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    # Sometimes images also have an alpha channel indicating transparency, but our network only wants RGB input.
    img_t = img_t[:3]
    batch[i] = img_t

### 4.1.4 Normalizing the data
- **Neural networks exhibit the best training performance when the input data ranges roughly from 0 to 1, or from -1 to 1** (this is an effect of how their building blocks are defined).
- So a typical thing we'll want to do is cast a tensor to floating-point and normalize the values of the pixels.
    - Normalization is trickier, as it depends on what range of the input we decide should lie between 0 and 1 (or -1 and 1).

In [8]:
batch = batch.float()
batch /= 255.0

- **Another possibility** is to **compute the mean and standard deviation of the input data and scale it so that the output has zero mean and unit standard deviation across each channel**:

In [9]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

- In working with images, it is good practice to compute the mean and standard deviation **on all the training data in advance and then subtract and divide by these fixed, precomputed quantities**. 